# Project 2 - Maps

Data from: Heyes, Anthony, and Soodeh Saberian. 2019. "Temperature and Decisions: Evidence from 207,000 Court Cases." American Economic Journal: Applied Economics, 11 (2): 238–65.

Notebooks used troughout the code: 
- ECO225 - Mapping Notebook
- Dictionaries mapping based on data found online, as outlined.

In [ ]:
!pip install geopandas plotly
!pip install geopandas geopy
!pip install cartopy
!pip install shapely

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely

In [ ]:
#Obtain shape file for US
state_df = gpd.read_file("http://www2.census.gov/geo/tiger/GENZ2016/shp/cb_2016_us_state_5m.zip")
state_df = state_df.rename(columns={"NAME":"Province_State"})
state_df

In [ ]:
df = pd.read_stata('matched_corrected.dta')
unique_counts = df['id'].value_counts()
print(unique_counts)

In [ ]:
#Information from: https://www.infoplease.com/us/geography/latitude-and-longitude-us-and-canadian-cities
city_coordinates = {
    "NEW YORK": (40, -74),
    "MIAMI": (26, -80),
    "LOS ANGELES": (34, -118),
    "SAN FRANCISCO": (38, -122),
    "CHICAGO": (42, -88),
    "NEWARK": (41, -74),
    "BALTIMORE": (39, -77),
    "ORLANDO": (29, -81),
    "ARLINGTON": (33, -97),
    "PHILADELPHIA": (40, -75),
    "BOSTON": (42, -71),
    "SAN DIEGO": (33, -117),
    "HOUSTON": (30, -95),
    "MEMPHIS": (35, -90),
    "DALLAS": (33, -97),
    "DETROIT": (42, -83),
    "DENVER": (40, -105),
    "ATLANTA": (33, -85),
    "HARTFORD": (42, -73),
    "LAS VEGAS": (36, -115),
    "SEATTLE": (48, -122),
    "PHOENIX": (33, -112),
    "NEW ORLEANS": (30, -90),
    "EL PASO": (32, -106),
    "SAN PEDRO": (34, -118),
    "HONOLULU": (21, -158),
    "SAN ANTONIO": (29, -98),
    "BUFFALO": (43, -79),
    "LANCASTER": (40, -76),
    "PORTLAND": (46, -123),
    "BRADENTON": (27, -83),
    "YORK": (40, -77),
    "IMPERIAL": (33, -116),
    "GUAYNABO": (18, -66),
    "TUCSON": (32, -111),
}

coordinates_df = pd.DataFrame.from_dict(city_coordinates, orient='index', columns=['latitude', 'longitude'])
coordinates_df.index.name = 'id'
coordinates_df.reset_index(inplace=True)
df_merged = pd.merge(df, coordinates_df, on='id', how='left')
print(df_merged)

In [ ]:
#Calculate the average resolution and temperature per city
city_avg = df_merged.groupby('city').agg(
    avg_res=('res', 'mean'),
    avg_temp=('temp6t4', 'mean'),
    lat=('latitude', 'first'),
    lon=('longitude', 'first')
).reset_index()

print(city_avg)

In [ ]:
city_state_mapping = {
    "NEW YORK": "NY",
    "MIAMI": "FL",
    "LOS ANGELES": "CA",
    "SAN FRANCISCO": "CA",
    "CHICAGO": "IL",
    "NEWARK": "NJ",
    "BALTIMORE": "MD",
    "ORLANDO": "FL",
    "ARLINGTON": "TX",
    "PHILADELPHIA": "PA",
    "BOSTON": "MA",
    "SAN DIEGO": "CA",
    "HOUSTON": "TX",
    "MEMPHIS": "TN",
    "DALLAS": "TX",
    "DETROIT": "MI",
    "DENVER": "CO",
    "ATLANTA": "GA",
    "HARTFORD": "CT",
    "LAS VEGAS": "NV",
    "SEATTLE": "WA",
    "PHOENIX": "AZ",
    "NEW ORLEANS": "LA",
    "EL PASO": "TX",
    "SAN PEDRO": "CA",
    "HONOLULU": "HI",
    "SAN ANTONIO": "TX",
    "BUFFALO": "NY",
    "LANCASTER": "PA",
    "PORTLAND": "OR",
    "BRADENTON": "FL",
    "YORK": "PA",
    "IMPERIAL": "CA",
    "GUAYNABO": "PR",  
    "TUCSON": "AZ",
}

city_state_df = pd.DataFrame(list(city_state_mapping.items()), columns=['city', 'state'])
city_avg = city_avg.merge(city_state_df, on='city', how='left')
print(city_avg)

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))
map_color = 'white'  
point_color = 'red'  

state_df.boundary.plot(ax=axs, linewidth=1, color='black')
state_df.plot(ax=axs, color="white", edgecolor='white')  # Fill states

# Plot each city as a pin
for lat, lon in city_coordinates.values():
    axs.plot(lon, lat, marker='D', color=point_color, markersize=10)

axs.set_xlim(-130, -60)
axs.set_ylim(20, 50)
axs.set_axis_off()
axs.grid(color='white', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
state_avg = city_avg.groupby('state').agg(
    avg_resolution=('avg_res', 'mean'),
    avg_temperature=('avg_temp', 'mean')
).reset_index()

print(state_avg)

In [ ]:
state_df = state_df.merge(state_avg, left_on='STUSPS', right_on='state', how='left')

In [ ]:
#Remove islands for better visibility of map
state_df = state_df[~state_df['Province_State'].isin(['Alaska', 'Hawaii','American Samoa','Guam',
                                                      'United States Virgin Islands',
                                                      'Commonwealth of the Northern Mariana Islands'])]

In [ ]:
#Map for average resolution
fig, rax = plt.subplots(figsize=(50, 20)) 

state_df.plot(ax=rax, edgecolor="black", color="white")

rax.set_xlabel('longitude')
rax.set_ylabel('latitude')

rax.spines['top'].set_visible(False) 
rax.spines['right'].set_visible(False) 
plt.axis('off')

state_df.plot(
     ax=rax, edgecolor='black', column='avg_resolution', 
     legend=True, cmap='PuBu', 
     vmin=0, vmax=1)

rax.annotate('Resolution Rate', xy=(0.92, 0.008), xycoords='axes fraction', fontsize=30)
plt.show()

In [ ]:
#Map for average temperature
fig, tax = plt.subplots(figsize=(50, 20)) 

state_df.plot(ax=tax, edgecolor="black", color="white")

tax.set_xlabel('longitude')
tax.set_ylabel('latitude')

tax.spines['top'].set_visible(False) 
tax.spines['right'].set_visible(False) 
plt.axis('off')

state_df.plot(
     ax=tax, edgecolor='black', column='avg_temperature', 
     legend=True, cmap='PuBu', 
     vmin=20, vmax=100)

tax.annotate('Temperature', xy=(0.95, 0.008), xycoords='axes fraction', fontsize=30)

plt.show()

## Republican vs Democract

In [ ]:
#Results 20000 Elections - https://www.archives.gov/electoral-college/2000
data = {
    "State": [
        "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
        "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia", 
        "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", 
        "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
        "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
        "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", 
        "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", 
        "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Washington", 
        "West Virginia", "Wisconsin", "Wyoming"
    ],
    "Republican": [
        1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,  
        1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,  
        1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,  
        0, 0, 0, 1, 1, 1, 1, 0, 0, 1,  
        1, 1, 1, 1, 0, 0  
    ]
}

df_data = pd.DataFrame(data)
state_df = state_df.merge(df_data, left_on='Province_State', right_on='State', how='left')
state_df

In [ ]:
# Calculate the average and standard deviation 
summary = state_df.groupby('Republican')['avg_resolution'].agg(['mean', 'std']).reset_index()
summary['Party'] = summary['Republican'].replace({1: 'Republican', 0: 'Democract'})

plt.figure(figsize=(7, 7), facecolor='white')  
ax = plt.subplot(111, facecolor='white')  
bars = plt.bar(summary['Party'], summary['mean'], yerr=summary['std'], capsize=5, color=['skyblue', 'lightcoral'])
plt.ylabel('Average Resolution')
plt.xlabel('Party Affiliation')
for spine in ax.spines.values():
    spine.set_visible(False)

plt.xticks(rotation=0)  
plt.tight_layout()       
plt.show()